# Import Libraries

In [37]:
from math import sqrt
import numpy as np
import warnings
from collections import Counter
import pandas as pd
import random

# Calculate Euclidean Distance

In [66]:
def k_nearest_neighbors (data, predict, k=3):
    if len(data) >= k:
        warning.warn('k is set to a value less than the total number of voting groups!')
    distances = []
    for group in data:
        for features in data[group]:
            #ecludiean distance can be gotten using either of the following:
            #euclidean_distance = sqrt( (features[0]-predict[0])**2 + (features[1]-predict[1])**2)
            #euclidean_distance = np.sqrt( np.sum((np.array(features) - np.array(predict))**2))
            euclidean_distance = np.linalg.norm(np.array(features) - np.array(predict)) #this is the faster of the 3
            distances.append([euclidean_distance, group])
    vote = [i[1] for i in sorted(distances)[:k]]
    #print(Counter(vote).most_common(1)) # comment this out to print only accuracy and not individual predictions
    vote_result = Counter(vote).most_common(1)[0][0]
    confidence = Counter(vote).most_common(1)[0][1]/k #creating confidence
    #print(vote_result, confidence) #uncomment to print vote and confidence
    return vote_result, confidence

# Import Dataset

In [67]:
df = pd.read_csv('breast-cancer-wisconsin.data') #Load data

In [68]:
df.head() #Display first 5 rows

,id,clump_thickness,Unif_cell_size,Unif_cell_shape,marg_adhesion,single_epith_cell,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


# Replace Missing Data and Drop Irrelivant Column

In [69]:
df.replace('?', -99999, inplace=True) #replace missing data with -99999 this is a very large outlier to avoid conflit
df.drop(['id'], 1, inplace=True) #drop the id column as it doesnt contribute to our prediction

# Cast all variables to float

In [70]:
data = df.astype(float).values.tolist() #to convert all values to float
data[:10]

[[5.0, 1.0, 1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0],
 [5.0, 4.0, 4.0, 5.0, 7.0, 10.0, 3.0, 2.0, 1.0, 2.0],
 [3.0, 1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 1.0, 1.0, 2.0],
 [6.0, 8.0, 8.0, 1.0, 3.0, 4.0, 3.0, 7.0, 1.0, 2.0],
 [4.0, 1.0, 1.0, 3.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0],
 [8.0, 10.0, 10.0, 8.0, 7.0, 10.0, 9.0, 7.0, 1.0, 4.0],
 [1.0, 1.0, 1.0, 1.0, 2.0, 10.0, 3.0, 1.0, 1.0, 2.0],
 [2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0],
 [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 5.0, 2.0],
 [4.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0]]

# Shuffle Dataset

In [71]:
#shuffle data
random.shuffle(data)
print(data[:10])

[[3.0, 1.0, 1.0, 3.0, 8.0, 1.0, 5.0, 8.0, 1.0, 2.0], [4.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0], [8.0, 10.0, 10.0, 1.0, 3.0, 6.0, 3.0, 9.0, 1.0, 4.0], [8.0, 4.0, 10.0, 5.0, 4.0, 4.0, 7.0, 10.0, 1.0, 4.0], [2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0], [5.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0], [6.0, 5.0, 5.0, 8.0, 4.0, 10.0, 3.0, 4.0, 1.0, 4.0], [8.0, 4.0, 4.0, 1.0, 6.0, 10.0, 2.0, 5.0, 2.0, 4.0], [6.0, 8.0, 8.0, 1.0, 3.0, 4.0, 3.0, 7.0, 1.0, 2.0], [5.0, 4.0, 5.0, 1.0, 8.0, 1.0, 3.0, 6.0, 1.0, 2.0]]


# Split Data to Training and Test Dataset

In [72]:
#Split train-test
test_size = 0.2
train_set = {2:[], 4:[]}
test_set = {2:[], 4:[]}
train_data = data[:-int(test_size*len(data))] #split train data = from beginning to the last 20% of the data
test_data = data[-int(test_size*len(data)):] #split test data = last 20% of the data

# populate the train set and test set dictionaries

In [73]:
for i in train_data:
    train_set[i[-1]].append(i[:-1]) #exclude the class column, which is what we are predicting
    
for i in test_data:
    test_set[i[-1]].append(i[:-1]) #exclude the class column, which is what we are predicting
    

# Pass the information into k_nearest_neighbors 

In [75]:


correct = 0
total = 0

for group in test_set:
    for data in test_set[group]:
        vote, confidence = k_nearest_neighbors (train_set, data, k=5) #increasing k increases the accuracy at a point then drops as k becomes too high. so adding to k might not increase the model 
        if group == vote:
            correct += 1
        else:
            print(confidence) # gives confidence of votes that are incorrect
        total += 1
print ('Accuracy:', correct/total) # i.e number of correct out of total

0.8
0.8
0.6
0.6
1.0
0.6
Accuracy: 0.9568345323741008
